In [ ]:
import random
import gc
import pickle
import numpy as np
import tensorflow as tf
from tensorflow import keras
import re
from typing import Tuple
from sklearn.preprocessing import LabelEncoder

def load_file(file_path: str) -> Tuple[Tuple[str], Tuple[str]]:
    """loads the test file and extracts all functions/derivatives"""
    data = open(file_path, "r").readlines()
    functions, derivatives = zip(*[line.strip().split("=") for line in data])
    return functions, derivatives

max_sequence_len = 32
batch_size = 512  # Batch size for training.
epochs = 10  # Number of epochs to train for.
latent_dim = 1024  # Latent dimensionality of the encoding space.
num_samples = 990000  # Number of samples to train on.
# Path to the data txt file on disk.
filepath = "sample_train.txt"

functions = []
derivatives = []
input_characters = set()
# target_characters = set()
with open(filepath, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in list(filter(None,lines[: min(num_samples, len(lines) - 1)])):
    function, derivative = line.split("=")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    derivative = "\t" + derivative + "\n"
    functions.append(function)
    derivatives.append(derivative)
    for char in list(filter(None, re.split(r"(exp|sin|cos|\^|\/|[a-zA-Z]|[0-9]|[()]|\+|\-|\*|\#)", function))):
        if char not in input_characters:
            input_characters.add(char)
    # for char in target_text:
    #     if char not in target_characters:
    #         target_characters.add(char)


input_characters.add('\n')
input_characters.add('\t')
input_characters.add('')
input_characters = sorted(list(input_characters))
# target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
# num_decoder_tokens = len(target_characters)
# max_encoder_seq_length = max([len(txt) for txt in input_texts])
# max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(functions))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_encoder_tokens)
print("Max sequence length for inputs:", max_sequence_len)
print("Max sequence length for outputs:", max_sequence_len)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])

encoder_input_data = np.zeros(
    (len(functions), max_sequence_len, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(functions), max_sequence_len, num_encoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(functions), max_sequence_len, num_encoder_tokens), dtype="float32"
)

for i, (function, derivative) in enumerate(zip(functions, derivatives)):
    for t, char in enumerate(list(filter(None,re.split(r"(exp|sin|cos|\^|\/|[a-zA-Z]|[0-9]|[()]|\+|\-|\*|\#|\n|\t)", function)))):
            encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index['']] = 1.0
    for t, char in enumerate(list(filter(None,re.split(r"(exp|sin|cos|\^|\/|[a-zA-Z]|[0-9]|[()]|\+|\-|\*|\#|\n|\t)", derivative)))):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, input_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, input_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, input_token_index['']] = 1.0
    decoder_target_data[i, t:, input_token_index['']] = 1.0
print('Feature vectors created successfully!!')

# print(decoder_input_data[0])


In [ ]:
"""
## Build the model
"""

# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# del encoder_outputs
# gc.collect()
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_encoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_encoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [ ]:
"""
## Train the model
"""

model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s_990000_latent_1024")


In [ ]:
model.save("s2s_990000_latent_1024")


In [ ]:
filepath_test = "sample_test.txt"
num_samples_test = 10000
functions_test = []
derivatives_test = []
# input_characters = set()
# target_characters = set()
with open(filepath_test, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in list(filter(None,lines[: min(num_samples_test, len(lines) - 1)])):
    function, derivative = line.split("=")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    # derivative = "\t" + derivative + "\n"
    functions_test.append(function)
    derivatives_test.append(derivative)
    # for char in list(filter(None, re.split(r"(exp|sin|cos|\^|\/|[a-zA-Z]|[0-9]|[()]|\+|\-|\*|\#)", function))):
    #     if char not in input_characters:
    #         input_characters.add(char)
    # for char in target_text:
    #     if char not in target_characters:
    #         target_characters.add(char)


# input_characters.add('\n')
# input_characters.add('\t')
# input_characters.add('')
# input_characters = sorted(list(input_characters))
# target_characters = sorted(list(target_characters))
# num_encoder_tokens = len(input_characters)
# num_decoder_tokens = len(target_characters)
# max_encoder_seq_length = max([len(txt) for txt in input_texts])
# max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of test samples:", len(functions_test))
# print("Number of unique input tokens:", num_encoder_tokens)
# print("Number of unique output tokens:", num_encoder_tokens)
# print("Max sequence length for inputs:", max_sequence_len)
# print("Max sequence length for outputs:", max_sequence_len)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])

encoder_input_data_test = np.zeros(
    (len(functions_test), max_sequence_len, num_encoder_tokens), dtype="float32"
)
# decoder_input_data = np.zeros(
#     (len(functions), max_sequence_len, num_encoder_tokens), dtype="float32"
# )
# decoder_target_data = np.zeros(
#     (len(functions), max_sequence_len, num_encoder_tokens), dtype="float32"
# )

for i, (function, derivative) in enumerate(zip(functions_test, derivatives_test)):
    for t, char in enumerate(list(filter(None,re.split(r"(exp|sin|cos|\^|\/|[a-zA-Z]|[0-9]|[()]|\+|\-|\*|\#|\n|\t)", function)))):
            encoder_input_data_test[i, t, input_token_index[char]] = 1.0
    encoder_input_data_test[i, t + 1 :, input_token_index['']] = 1.0
    # for t, char in enumerate(list(filter(None,re.split(r"(exp|sin|cos|\^|\/|[a-zA-Z]|[0-9]|[()]|\+|\-|\*|\#|\n|\t)", derivative)))):
    #     # decoder_target_data is ahead of decoder_input_data by one timestep
    #     decoder_input_data[i, t, input_token_index[char]] = 1.0
    #     if t > 0:
    #         # decoder_target_data will be ahead by one timestep
    #         # and will not include the start character.
    #         decoder_target_data[i, t - 1, input_token_index[char]] = 1.0
    # decoder_input_data[i, t + 1 :, input_token_index['']] = 1.0
    # decoder_target_data[i, t:, input_token_index['']] = 1.0
print('Feature vectors for test sample created successfully!!')


In [ ]:
"""
## Run inference (sampling)
1. encode input and retrieve initial decoder state
2. run one step of decoder with this initial state
and a "start of sequence" token as target.
Output will be the next target token.
3. Repeat with the current target token and current states
"""

# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s_990000_latent_1024")
print('Model loaded successfully')
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)
print('decoder model created!!')
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in input_token_index.items())

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq, verbose=0)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_encoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, input_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value, verbose=0)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_sequence_len:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_encoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence




In [ ]:
"""
You can now generate decoded sentences as such:
"""
match = 0
for seq_index in range(num_samples_test):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data_test[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    # print("-")
    # print("Input sentence:", functions_test[seq_index])
    # print("Decoded sentence:", decoded_sentence.strip().replace('\t','').replace('\n',''))
    # print("actual derivative:",str(derivatives_test[seq_index]).strip().replace('\t','').replace('\n',''))
    if(decoded_sentence.strip().replace('\t','').replace('\n','')
       == str(derivatives_test[seq_index]).strip().replace('\t','').replace('\n','')):
        match += 1
    if(seq_index%10==0):
        print(str(match)+'/'+str(seq_index))

print("accuracy is : ",match/num_samples_test)

In [2]:
from random import shuffle

with open("sample_test.txt", 'r') as f:
    lines = f.read().split("\n")
    shuffle(lines)
    with open("test.txt", 'w') as fp:
        fp.write('\n'.join(lines[:100]))
